<a href="https://colab.research.google.com/github/masoudshahrian/MachineLearning-Code/blob/master/Image_Clustring_using_GPC_Q_learning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np
import cv2
from sklearn.cluster import KMeans
import random


In [2]:
from google.colab.patches import cv2_imshow

In [3]:
class QLearningGPC:
    def __init__(self, n_clusters=3, alpha=0.1, gamma=0.9, epsilon=0.1):
        self.n_clusters = n_clusters
        self.alpha = alpha
        self.gamma = gamma
        self.epsilon = epsilon
        self.q_table = np.zeros((n_clusters, n_clusters))

    def select_action(self, state):
        if random.uniform(0, 1) < self.epsilon:
            return random.randint(0, self.n_clusters - 1)
        else:
            return np.argmax(self.q_table[state])

    def update_q_table(self, state, action, reward, next_state):
        best_next_action = np.argmax(self.q_table[next_state])
        self.q_table[state, action] += self.alpha * (reward + self.gamma * self.q_table[next_state, best_next_action] - self.q_table[state, action])

    def fit(self, image):
        img_reshape = image.reshape((-1, 3))
        kmeans = KMeans(n_clusters=self.n_clusters).fit(img_reshape)
        labels = kmeans.labels_

        for _ in range(100):
            for i in range(len(labels)):
                state = labels[i]
                action = self.select_action(state)
                reward = -np.linalg.norm(img_reshape[i] - kmeans.cluster_centers_[action])
                next_state = action
                self.update_q_table(state, action, reward, next_state)

        for i in range(len(labels)):
            labels[i] = self.select_action(labels[i])

        segmented_image = np.zeros_like(img_reshape)
        for i in range(len(labels)):
            segmented_image[i] = kmeans.cluster_centers_[labels[i]]

        return segmented_image.reshape(image.shape)

# Load an example image
image = cv2.imread('/content/4.jpg')
image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

# Apply QLearningGPC
qlearning_gpc = QLearningGPC(n_clusters=3)
segmented_image = qlearning_gpc.fit(image)

# Save and display results
cv2.imwrite('segmented_image.jpg', cv2.cvtColor(segmented_image.astype(np.uint8), cv2.COLOR_RGB2BGR))
# cv2_imshow('Segmented Image',segmented_image.astype(np.uint8))


True